In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime, date

In [2]:
# Random seed for reproducibility
np.random.seed(42)

# EUgENia

## Generando Informacion SIntetica

### Inventario Sintetico

##### Hipotesis:

0. Los bienes son no perecederos
1. Se asume se conoce en promedio cuantas unidades se venden a la semana. En este caso se modela con una funcion de probabilidad uniforme
2. Con base en el promedio de unidades que se venden a la semana, se sabe para cuantas semanas hay inventario

In [3]:
# Create the parameters for the data
n_brands = 15  # Number of brands
skus_per_brand = 100  # SKUs per brand
min_inventory_days = 7.0  # Minimum days of inventory
max_inventory_days = 14.0  # Maximum days of inventory

# Sales frequency settings
sales_settings = {
    "low": {"min_daily_sales": 1, "max_daily_sales": 4},
    "medium": {"min_daily_sales": 4, "max_daily_sales": 8},
    "high": {"min_daily_sales": 8, "max_daily_sales": 12}
}

brands = [f"Marca_{i+1}" for i in range(n_brands)]
# Define specific store allocations for brands
brand_stores = {f"Marca_{i+1}": 20 for i in range(3)}  # Explicitly create keys for Marca_1 to Marca_3
# Distribute the remaining stores across the other brands
remaining_brands = [brand for brand in brands if brand not in brand_stores]
remaining_stores = 148 - sum(brand_stores.values())
stores_per_remaining_brand = remaining_stores // len(remaining_brands)

for brand in remaining_brands:
    brand_stores[brand] = stores_per_remaining_brand

# Account for any rounding errors in distribution
additional_stores = remaining_stores % len(remaining_brands)
for i in range(additional_stores):
    brand_stores[remaining_brands[i]] += 1

# Generate stores based on the brand they belong to
stores = []
for brand, count in brand_stores.items():
    stores.extend([f"Tienda_{i+1}_{brand}" for i in range(count)])
# Generate SKUs
skus = {brand: [f"SKU_{i+1}_{brand}" for i in range(skus_per_brand)] for brand in brands}

products = []
for store in stores:
    # Split the store name and extract the last part for the brand name
    l = store.split('_')  # This isolates 'Marca_1', 'Marca_2', etc.
    brand = l[2] + "_" + l[3]
    for sku in skus[brand]:  # Now 'brand' should be correctly formatted like "Marca_1"
        index = int(sku.split('_')[1]) % 3  # Assuming SKU format is "SKU_{index}_{brand}"
        if index == 0:
            freq_type = "low"
        elif index == 1:
            freq_type = "medium"
        else:
            freq_type = "high"

        min_daily_sales = sales_settings[freq_type]["min_daily_sales"]
        max_daily_sales = sales_settings[freq_type]["max_daily_sales"]

        avg_daily_sales = round(np.random.uniform(min_daily_sales, max_daily_sales))
        days_inventory = round(np.random.uniform(min_inventory_days, max_inventory_days))
        units_in_inventory = round(avg_daily_sales * days_inventory)
        products.append({
            "SKU": sku,
            "Marca": brand,
            "Tienda": store,
            "Promedio Unidades Vendidas Diariamente": avg_daily_sales,
            "Numero Dias Inventario Restante": days_inventory,
            "Unidades en Inventario": units_in_inventory,
            "Frecuencia": freq_type  # Add frequency type for reference
        })

# Generate the DataFrame
inventario_df = pd.DataFrame(products)

# Add a new column "Fecha" with today's date
inventario_df['Fecha Corte Inventario'] = date.today()

In [4]:
inventario_df.head(1)

,SKU,Marca,Tienda,Promedio Unidades Vendidas Diariamente,Numero Dias Inventario Restante,Unidades en Inventario,Frecuencia,Fecha Corte Inventario
0,SKU_1_Marca_1,Marca_1,Tienda_1_Marca_1,5,14,70,medium,2024-05-20


In [5]:
len(set(inventario_df['Tienda']))

148

In [6]:
inventario_df.to_csv('/home/ricmunrom/Documents/jorgeLozano/EUgeNia/primeraExploracion/inputsSinteticos/inventario.csv', index = False)

### Demanda Esperada Sintetica

##### Hipotesis:

0. Se utiliza como lambda de la distribucion poisson el Promedio Unidades Venidas Diariamente para simular la demanda esperada diaria para cada SKU

In [7]:
# Parameters and DataFrame setup
n_brands = 15
skus_per_brand = 100
days = 32  # Number of days (1 month - four weeks)

# Create date range for 3 months starting from today
start_date = date.today()
date_range = [start_date + timedelta(days=i) for i in range(days)]

# Generate expected demand DataFrame
demand_data = []
for index, row in inventario_df.iterrows():
    demands = np.random.poisson(lam=row["Promedio Unidades Vendidas Diariamente"], size=days)
    for date_col, demand in zip(date_range, demands):
        demand_data.append({
            "Fecha": date_col,
            "SKU": row["SKU"],
            "Marca": row["Marca"],
            "Tienda": row["Tienda"],
            "Demanda Esperada": demand
        })

# Create DataFrame
demanda_esperada_df = pd.DataFrame(demand_data)

# Reorder columns for better readability
demanda_esperada_df = demanda_esperada_df[["Fecha", "SKU", "Marca", "Tienda", "Demanda Esperada"]]

In [8]:
demanda_esperada_df.head(1)

,Fecha,SKU,Marca,Tienda,Demanda Esperada
0,2024-05-20,SKU_1_Marca_1,Marca_1,Tienda_1_Marca_1,3


In [9]:
demanda_esperada_df.tail(1)

,Fecha,SKU,Marca,Tienda,Demanda Esperada
473599,2024-06-20,SKU_100_Marca_15,Marca_15,Tienda_7_Marca_15,11


In [10]:
demanda_esperada_df.shape

(473600, 5)

In [11]:
demanda_esperada_df.to_csv('/home/ricmunrom/Documents/jorgeLozano/EUgeNia/primeraExploracion/inputsSinteticos/demandaEsperada.csv', index = False)

### CEDI sintetico

##### Hipotesis

Existe un unico Centro de Distribucion donde se reciben los volumenes solicitados por las ordenes de compra. Se asume que el punto de partida del inventario en CEDI de cada SKU es una cuarta parte de la suma de todas las unidades en Inventario entre todas las tiendas de ese SKU

In [12]:
# Ensure all_skus list is available; otherwise, generate it from inventario_df
all_skus = inventario_df['SKU'].unique()

# Initialize the list to store the data for the new DataFrame
cedi_data = []

# Iterate over each SKU to calculate the mean units in inventory
for sku in all_skus:
    # Find the sum units in inventory of all stores for the SKU
    sum_units = inventario_df[inventario_df['SKU'] == sku]['Unidades en Inventario'].sum()
    
    # Generate a random number around this sum divided by nothing (you might adjust the method based on your requirements)
    unidades = np.random.poisson(lam=sum_units)
    
    # Append the data to the list
    cedi_data.append({
        "SKU": sku,
        "Unidades Disponible": unidades,
        "Fecha": date.today()
    })

# Create DataFrame from the data
CEDI_df = pd.DataFrame(cedi_data)

#CEDI_df = pd.pivot_table(CEDI_df, values = 'Unidades Disponible', index= 'SKU', columns = 'Fecha')

In [13]:
CEDI_df

,SKU,Unidades Disponible,Fecha
0,SKU_1_Marca_1,1191,2024-05-20
1,SKU_2_Marca_1,2098,2024-05-20
2,SKU_3_Marca_1,515,2024-05-20
3,SKU_4_Marca_1,1360,2024-05-20
4,SKU_5_Marca_1,2008,2024-05-20
...,...,...,...
1495,SKU_96_Marca_15,197,2024-05-20
1496,SKU_97_Marca_15,433,2024-05-20
1497,SKU_98_Marca_15,616,2024-05-20
1498,SKU_99_Marca_15,142,2024-05-20


In [14]:
CEDI_df.to_csv('/home/ricmunrom/Documents/jorgeLozano/EUgeNia/primeraExploracion/inputsSinteticos/CEDI.csv', index = False)

### Lead Times Sinteticos

##### Hipotesis:

0. Se deben generar los mismos SKUs para las mismas marcas, resultando en 80 SKUs diferentes (8 SKUs por 10 marcas).
1. Se deben generar al menos 100 órdenes de compra.
2. Cada orden de compra debe incluir una selección aleatoria de entre 33 y 66 SKUs de los 80 disponibles.
3. La fecha de la orden de compra para cada orden debe ser una fecha aleatoria.
4. La fecha de recepción de mercancía para cada orden debe ser también una fecha aleatoria, pero posterior a la fecha de la orden de compra.
5. La columna de "Lead Time" debe ser un número aleatorio entero entre 7 y 31 días y debe corresponder a la diferencia en días entre la fecha de orden de compra y la fecha de recepción de mercancía.
6. El "Volumen Solicitado" para cada SKU en cada orden de compra debe determinarse usando una función de distribución de Poisson, con un lambda igual al doble del valor encontrado en "Promedio Unidades Vendidas Semanalmente" para ese SKU en `inventario_df`.
7. El "Volumen Recibido" para cada SKU en cada orden de compra debe ser un número aleatorio entre 0 y el "Volumen Solicitado".
8. La "Fiabilidad" se calcula como la división del "Volumen Recibido" entre el "Volumen Solicitado", multiplicado por 100 para obtener un porcentaje.

In [15]:
# Suponiendo que demanda_esperada_df, inventario_df y CEDI_df ya están definidos

# Set the order date to 2024-05-13
order_date = datetime(2024, 5, 13)

# Calculate the total expected demand for each SKU across all stores
total_demand_per_sku = demanda_esperada_df.groupby("SKU")["Demanda Esperada"].sum().reset_index()

# Merge with inventory data to get total inventory per SKU across all stores
total_inventory_per_sku = inventario_df.groupby("SKU")["Unidades en Inventario"].sum().reset_index()
total_demand_per_sku = total_demand_per_sku.merge(total_inventory_per_sku, on="SKU", how="left").fillna(0)

# Merge with CEDI inventory data to get total inventory in CEDI for each SKU
total_demand_per_sku = total_demand_per_sku.merge(CEDI_df[["SKU", "Unidades Disponible"]], on="SKU", how="left").fillna(0)

# Adjust demand based on inventory levels
total_demand_per_sku["Adjusted Demand"] = total_demand_per_sku["Demanda Esperada"] - total_demand_per_sku["Unidades en Inventario"] - total_demand_per_sku["Unidades Disponible"]
total_demand_per_sku["Adjusted Demand"] = total_demand_per_sku["Adjusted Demand"].apply(lambda x: max(x, 0))  # Ensure no negative demand

# Verify adjusted demands
print(total_demand_per_sku[['SKU', 'Demanda Esperada', 'Unidades en Inventario', 'Unidades Disponible', 'Adjusted Demand']].head())

# Generate the purchase order
orders_data = []

for index, row in total_demand_per_sku.iterrows():
    sku = row["SKU"]
    adjusted_demand = row["Adjusted Demand"]
    lead_time_days = np.random.randint(1, 4)  # Lead time between 1 and 3 days
    reception_date = order_date + timedelta(days=lead_time_days)
    volume_requested = adjusted_demand
    # Assuming 100% reliability
    #volume_received = volume_requested
    # Assuming reliability at 90%
    volume_received = int(volume_requested * 0.90 + np.random.uniform(-0.1, 0.1) * volume_requested)  # Adjust to achieve average reliability of 90%
    reliability = (volume_received / volume_requested) * 100 if volume_requested > 0 else 0

    orders_data.append({
        "SKU": sku,
        "Marca": sku.split('_')[2] + '_' + sku.split('_')[3],  # Correct extraction of brand from SKU
        "ID Orden de Compra": "OC_1",
        "Fecha Orden de Compra": order_date,
        "Fecha Recepcion Mercancia": reception_date,
        "Lead Time en Dias": lead_time_days,
        "Volumen Solicitado": volume_requested,
        "Volumen Recibido": volume_received,
        "Fiabilidad": reliability
    })

ordenes_compra_df = pd.DataFrame(orders_data)

                SKU  Demanda Esperada  Unidades en Inventario  \
0   SKU_100_Marca_1              3637                    1194   
1  SKU_100_Marca_10              1602                     548   
2  SKU_100_Marca_11              1432                     399   
3  SKU_100_Marca_12              1380                     553   
4  SKU_100_Marca_13              1527                     477   

   Unidades Disponible  Adjusted Demand  
0                 1191             1252  
1                  503              551  
2                  371              662  
3                  557              270  
4                  462              588  


In [16]:
ordenes_compra_df

,SKU,Marca,ID Orden de Compra,Fecha Orden de Compra,Fecha Recepcion Mercancia,Lead Time en Dias,Volumen Solicitado,Volumen Recibido,Fiabilidad
0,SKU_100_Marca_1,Marca_1,OC_1,2024-05-13,2024-05-14,1,1252,1207,96.405751
1,SKU_100_Marca_10,Marca_10,OC_1,2024-05-13,2024-05-15,2,551,468,84.936479
2,SKU_100_Marca_11,Marca_11,OC_1,2024-05-13,2024-05-14,1,662,593,89.577039
3,SKU_100_Marca_12,Marca_12,OC_1,2024-05-13,2024-05-14,1,270,222,82.222222
4,SKU_100_Marca_13,Marca_13,OC_1,2024-05-13,2024-05-14,1,588,498,84.693878
...,...,...,...,...,...,...,...,...,...
1495,SKU_9_Marca_5,Marca_5,OC_1,2024-05-13,2024-05-14,1,249,239,95.983936
1496,SKU_9_Marca_6,Marca_6,OC_1,2024-05-13,2024-05-15,2,202,186,92.079208
1497,SKU_9_Marca_7,Marca_7,OC_1,2024-05-13,2024-05-16,3,225,202,89.777778
1498,SKU_9_Marca_8,Marca_8,OC_1,2024-05-13,2024-05-14,1,156,134,85.897436


In [17]:
ordenes_compra_df['Fiabilidad'].mean()

90.14264274785427

In [18]:
ordenes_compra_df['Lead Time en Dias'].mean()

1.9853333333333334

In [19]:
ordenes_compra_df.to_csv('/home/ricmunrom/Documents/jorgeLozano/EUgeNia/primeraExploracion/inputsSinteticos/ordenesCompra.csv', index = False)